# 2. House prices model
In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

Load the houseprices data from Thinkful's database.
Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
Do you think your model is satisfactory? If so, why?
In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.
For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')



In [78]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [79]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprices_df = pd.read_sql_query('select * from houseprices', con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [113]:
houseprices_df.centralair

0       Y
1       Y
2       Y
3       Y
4       Y
5       Y
6       Y
7       Y
8       Y
9       Y
10      Y
11      Y
12      Y
13      Y
14      Y
15      Y
16      Y
17      Y
18      Y
19      Y
20      Y
21      Y
22      Y
23      Y
24      Y
25      Y
26      Y
27      Y
28      Y
29      N
       ..
1430    Y
1431    Y
1432    Y
1433    Y
1434    Y
1435    Y
1436    Y
1437    Y
1438    Y
1439    Y
1440    Y
1441    Y
1442    Y
1443    N
1444    Y
1445    Y
1446    Y
1447    Y
1448    N
1449    Y
1450    Y
1451    Y
1452    Y
1453    Y
1454    Y
1455    Y
1456    Y
1457    Y
1458    Y
1459    Y
Name: centralair, Length: 1460, dtype: object

In [81]:
houseprices_df1=houseprices_df #make a copy

In [89]:
houseprices_df1 = pd.concat([houseprices_df1,pd.get_dummies(houseprices_df1.mszoning, prefix="mszoning", drop_first=True)], axis=1)
houseprices_df1 = pd.concat([houseprices_df1,pd.get_dummies(houseprices_df1.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(houseprices_df1.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(houseprices_df1.street, prefix="street", drop_first=True).columns)


# Build your initial model using these features and estimate the parameters using OLS.

In [90]:
X = houseprices_df1[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf'] + dummy_column_names]
y = houseprices_df1.saleprice

In [91]:
import statsmodels.api as sm

X = sm.add_constant(X)
results = sm.OLS(y, X).fit()

In [92]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     482.0
Date:                Wed, 31 Jul 2019   Prob (F-statistic):               0.00
Time:                        19:58:37   Log-Likelihood:                -17475.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1449   BIC:                         3.503e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.173e+05    1.8e+04     -6.502      0.000   -1.53e+05   -8.19e+04
overallqual  2.333e+04   1088.506     21.430      0.000    2.12e+04    2.55e+04
grlivarea      45.6344      2.468     18.494      0.000      40.794      50.475
garagecars   1.345e+04   2990.453      4.498      0.000    7584.056    1.93e+04
garagearea     16.4082     10.402      1.577      0.115      -3.997      36.813
totalbsmtsf    28.3816      2.931      9.684      0.000      22.633      34.131
mszoning_FV  2.509e+04   1.37e+04      1.833      0.067   -1761.679    5.19e+04
mszoning_RH  1.342e+04   1.58e+04      0.847      0.397   -1.77e+04    4.45e+04
mszoning_RL  2.857e+04   1.27e+04      2.246      0.025    3612.782    5.35e+04
mszoning_RM  9047.7894   1.29e+04      0.702      0.483   -1.62e+04    3.43e+04
street_Pave   151.0376   1.63e+04      0.009      0.993   -3.18e+04    3.21e+04
==============================================================================
Omnibus:                      415.883   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            41281.526
Skew:                          -0.115   Prob(JB):                         0.00
Kurtosis:                      29.049   Cond. No.                     5.53e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Our R squared and Adj R squared 76.9% and 76.7% respectively. F stat is 482 with a significant p value of 0. Our AIC and BIC are 3.4 and 3.5. There's definitely room for improvement. Let's mix up our variables. 

In [114]:
houseprices_df1 = pd.concat([houseprices_df1,pd.get_dummies(houseprices_df1.utilities, prefix="utilities", drop_first=True)], axis=1)
dummy_column_names =list(pd.get_dummies(houseprices_df1.utilities, prefix="utilities", drop_first=True).columns)
houseprices_df1 = pd.concat([houseprices_df1,pd.get_dummies(houseprices_df1.centralair, prefix="centralair", drop_first=True)], axis=1)
dummy_column_names =dummy_column_names+list(pd.get_dummies(houseprices_df1.centralair, prefix="centralair", drop_first=True).columns)

In [115]:
houseprices_df1.unique()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,saleprice,utilities_NoSeWa,mszoning_FV,mszoning_RH,mszoning_RL,mszoning_RM,street_Pave,utilities_NoSeWa,utilities_NoSeWa,centralair_Y
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,208500,0,0,0,1,0,1,0,0,1
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,181500,0,0,0,1,0,1,0,0,1
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,223500,0,0,0,1,0,1,0,0,1
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,140000,0,0,0,1,0,1,0,0,1
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,250000,0,0,0,1,0,1,0,0,1
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,143000,0,0,0,1,0,1,0,0,1
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,307000,0,0,0,1,0,1,0,0,1
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,200000,0,0,0,1,0,1,0,0,1
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,129900,0,0,0,0,1,1,0,0,1
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,118000,0,0,0,1,0,1,0,0,1


In [116]:
X = houseprices_df1[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'mszoning_RL', 'lotarea', 'overallcond'] + dummy_column_names]
y = houseprices_df1.saleprice

In [117]:
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()

In [119]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     549.0
Date:                Wed, 31 Jul 2019   Prob (F-statistic):               0.00
Time:                        20:45:45   Log-Likelihood:                -17461.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1450   BIC:                         3.500e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.322e+05   7429.490    -17.791      0.000   -1.47e+05   -1.18e+05
overallqual       2.422e+04   1075.542     22.516      0.000    2.21e+04    2.63e+04
grlivarea           43.2135      2.494     17.329      0.000      38.322      48.105
garagecars        1.806e+04   1741.324     10.372      0.000    1.46e+04    2.15e+04
totalbsmtsf         27.5937      2.898      9.522      0.000      21.909      33.278
mszoning_RL       1.397e+04   2589.917      5.396      0.000    8893.600    1.91e+04
lotarea              0.5007      0.107      4.681      0.000       0.291       0.711
overallcond       3209.4764    933.713      3.437      0.001    1377.904    5041.049
utilities_NoSeWa -1.261e+04   1.27e+04     -0.996      0.319   -3.75e+04    1.22e+04
utilities_NoSeWa -1.261e+04   1.27e+04     -0.996      0.319   -3.75e+04    1.22e+04
utilities_NoSeWa -1.261e+04   1.27e+04     -0.996      0.319   -3.75e+04    1.22e+04
centralair_Y      4980.3470   4373.332      1.139      0.255   -3598.386    1.36e+04
==============================================================================
Omnibus:                      424.744   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44976.290
Skew:                          -0.152   Prob(JB):                         0.00
Kurtosis:                      30.189   Cond. No.                     1.89e+23
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.7e-36. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

We were able to improve the model slightly with different feature selections. Our R squared is now 77.3% and adj R squared is 77.2%. F statistic went up to 549 with a siginificant p value. The AIC and BIC remained the same. If I were to use a model, I would pick the 2nd one based on these metrics. s